In [ ]:
pip install gensim
pip install torch
pip install torch.utils

In [ ]:
import torch.utils.checkpoint
import gensim.models

In [ ]:
#Importing and Modifying Training & Testing Data


#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

#Modules
import gensim.models
import pandas as pd
import torch
import numpy as np
from gensim.models import FastText, Word2Vec, KeyedVectors

#Deepti Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/medical_tc_test.csv')

drive.mount("/content/drive", force_remount=True)
modelemb = Word2Vec.load('/content/drive/MyDrive/Senior Year/Deep Learning/Deep Learning Final Project/w2v_OA_CR_300d.bin')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

#In training dataset, make each class have 1100 abstract samples to even out classes
dataset = dataset[dataset[:, 0].argsort()]
datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]
  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]
dataset=datasetnew

In [ ]:
#Importing and Modifying Training & Testing Data

from google.colab import drive
drive.mount('/content/drive')

#modules
import pandas as pd
import torch
import numpy as np

from gensim.models import FastText, Word2Vec, KeyedVectors

#Deepti Import Data
dataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_train.csv')
testdataset = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/medical_tc_test.csv')
#wordembeddings = pd.read_csv('/content/drive/MyDrive/Deep Learning Project 2/w2v_100d_oa_all.tar.gz')

drive.mount("/content/drive", force_remount=True)

modelemb = Word2Vec.load('/content/drive/MyDrive/Deep Learning Project 2/w2v_OA_CR_300d.bin')
modelemb.wv.has_index_for('diabetes')

dataset = pd.DataFrame.to_numpy(dataset)
testdataset = pd.DataFrame.to_numpy(testdataset)

#In training dataset, make each class have 1100 abstract samples to even out classes
dataset = dataset[dataset[:, 0].argsort()]
datasetnew=np.copy(dataset[:5500,:])

Dofstarts=dict()
for i in range(len(dataset)):
  row=dataset[i]
  if int(row[0]) not in Dofstarts:
    Dofstarts[int(row[0])] = i

for i in range(5):
  i = i+1
  valtostart=Dofstarts[i]
  datasetnew[(i-1)*1100:i*1100,:] = dataset[valtostart:valtostart+1100,:]
dataset=datasetnew

In [ ]:
#Testing Tokenization

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

########## ADDED

import spacy
from torchtext.data.utils import get_tokenizer

# Create a custom tokenizer using spacy
basic_english_tokenizer = get_tokenizer('basic_english')

# Define a custom tokenizer that also splits on dashes
def custom_tokenizer(text):
    # Use the basic_english tokenizer and split on dashes
    tokens = basic_english_tokenizer(text)
    # Flatten the list of tokens and split on dashes
    tokens = [subtoken for token in tokens for subtoken in token.split("-")]
    return tokens

# Example usage
text = "This is a test-sentence with a dash."
tokens = custom_tokenizer(text)
tokenizer=custom_tokenizer

##############

traindata = dataset[:,1]
trainlabel=dataset[:,0]

#get the lengths of the training data abstracts
lengthsoftrain=[]
x,y = dataset.shape
min = 100000000000

for i in range(x):
  lengthsoftrain.append(len(tokenizer(traindata[i])))

testdata = testdataset[:,1]
testlabel=testdataset[:,0]

#get the lengths of the testing data abstracts
lengthsoftest=[]
x,y = testdataset.shape

old=get_tokenizer('basic_english')
min = 100000000000
for i in range(x):
  lengthsoftest.append(len(tokenizer(testdata[i])))


In [ ]:
#Generating Mask: tensor of "False" values of size WORDlen - base code for mask from stack overflow

WORDlen = 50 #In LSTM, we used WORDlen = 20

lengthsoftrain = torch.tensor(lengthsoftrain)
trainmask = torch.logical_not(torch.arange(WORDlen).expand(len(lengthsoftrain), WORDlen) < lengthsoftrain.unsqueeze(1))

lengthsoftest=torch.tensor(lengthsoftest)
testmask= torch.logical_not(torch.arange(WORDlen).expand(len(lengthsoftest), WORDlen) < lengthsoftest.unsqueeze(1))


In [ ]:
#Tokenizing Training & Testing Data

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#tokenizer = get_tokenizer('basic_english')

class CustomTextDataset(Dataset):
    def __init__(self, txt, labels,mask):
        self.labels = labels
        self.text = txt
        self.mask=mask

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]-1
        text = self.text[idx]

        text = tokenizer(text)
        text = np.array(text)

        #while np.shape(text)[0] < WORDlen:
            #text = np.append(text, text)

        text = text[:WORDlen] #WORDlen = 50

        while np.shape(text)[0] < WORDlen:
            text = np.append(text, ["hjgjhgjhgfgdgfdgfdfjhj"])

        text = list(text)
        mask = self.mask[idx] #masking the abtract text

        for wordi in range(len(text)):
          word = text[wordi]
          if not modelemb.wv.has_index_for(word): #if no clinical embedding
            mask[wordi] = True #we mask these words

        sample = {"Text": text, "Class": label, "Mask": mask}
        return sample

#Get tokenized versions of the abstract samples in the training and testing where words without medical embeddings are masked
traindata_set = CustomTextDataset(traindata, trainlabel, trainmask)
testdata_set = CustomTextDataset(testdata, testlabel, testmask) #random think into it


In [ ]:
#Data Load --> getting shuffled batches for training

trainloader = DataLoader(traindata_set, batch_size = 64, shuffle = True)
trainloader2 = DataLoader(traindata_set, batch_size = 2000, shuffle = True) #WAS 4000
testloader2 = DataLoader(testdata_set, batch_size = 2000, shuffle = True)

#parameters
num_epochs = 50

#for epochs in range(num_epochs):
"""
for i in trainloader:
  print(i)

for idx,D in enumerate(trainloader):
    print(D)
    break
"""

for (idx, batch) in enumerate(trainloader):
    # Print the 'text' data of the batch
    print(np.array(batch['Text']).T[0])
    print(idx, 'Text data: ', (batch['Text'])[0][1])
    # Print the 'class' data of batch
    print(idx, 'Class data: ', len(batch['Class']), '\n')

    break

In [ ]:
#Building the Vocabulary
#Our current implementation of the vocabulary is all words that are seen in training and testing data.

def build_vocab(first, second, WORDlen): #WORDlen = 50 (from above)
  L = []

  if first: #words from train
    for i in traindata:
      L.append(tokenizer(i)[:WORDlen]) #get cropped tokenized abstract

  if second: #words from test
    for i in testdata:
      L.append(tokenizer(i)[:WORDlen]) #get cropped tokenized abstract

  L=np.array(L)
  L=np.concatenate(L)

  return list(L)

test_vocab_list=dict()
train_vocab_list=dict()
train_tokens = list(set(build_vocab(True, True, WORDlen)))

"""
for wordlength in [50,100]:
  z=(len(list(set(build_vocab(True,True,wordlength))))) #a+b+c
  x=(len(list(set(build_vocab(True,False,wordlength)))))  #a+c
  y=(len(list(set(build_vocab(False,True,wordlength))))) #b+c
  c=x+y-z
  print(y,y-c)
"""
"""
for wordlength in [50]:
  train_tokens_1 = list(set(build_vocab(True,False,wordlength)))
  test_tokens_1 = list(set(build_vocab(False,True,wordlength)))
  test_vocab_list = dict(zip(test_tokens_1, range(len(test_tokens_1))))
  train_vocab_list = dict(zip(train_tokens_1, range(len(train_tokens_1))))

count_test=0
count_train=0
total=0

for word in test_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_test+=1
  total+=1
  x = modelemb.wv.get_vector(word)


  #if word not in train_vocab_list:
  #  print(word)
for word in train_vocab_list:
  if not(modelemb.wv.has_index_for(word)): count_train+=1
  total+=1
print(count_test/total)
print(count_train/total)

"""

vocab_list = dict(zip(train_tokens, range(len(train_tokens))))

'''
vocab_list_one_hot = dict()
for i in vocab_list:
  length = len(vocab_list)
  one_hot = np.zeros(length)
  vocab_list[i]
  one_hot[vocab_list[i]] = 1
  vocab_list_one_hot[i] = one_hot'''


In [ ]:
#Transformer Implementation
#lambda trick and checkpoint from other students in OH

import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
dropout_ratio=0.3

class MyNeuralNet(nn.Module):
    def __init__(self):
        super(MyNeuralNet,self).__init__()

        self.positional_encoding = PositionalEncoding(d_model=300, dropout=0) #300 dimensional medical embeddings

        encoder_layers = TransformerEncoderLayer(d_model=300, nhead=1, batch_first=True, dropout=dropout_ratio)
        self.transformer_encoder = TransformerEncoder(encoder_layers, 1)

        #self.multihead_attn = nn.MultiheadAttention(embed_dim=300, num_heads=2)
        #self.transformer =nn.Transformer(d_model=300,nhead=2,batch_first=True)
        #nn.transformer
        #self.rnn = torch.nn.LSTM(300,100,1,batch_first=True,dropout=.2)
        self.flatten=nn.Flatten(1,2) #N, L, E
        inp=300

        self.dropout = nn.Dropout(.3)
        self.hid_layer = nn.Linear(inp*WORDlen, 1000)
        self.dropout2 = nn.Dropout(.3)
        out=inp*WORDlen
        #out=9000
        self.out_layer = nn.Linear(out, 5)
        self.sm = nn.Softmax(dim=1)


    def forward(self,x,mask):

        #x, attn_output_weights=self.multihead_attn(x,x,x)
        self.dummy_tensor = torch.ones(1, dtype=torch.float32, requires_grad=True)
        x=self.positional_encoding(x)
        x=torch.utils.checkpoint.checkpoint(lambda x,y: self.transformer_encoder(x,src_key_padding_mask=mask),x, self.dummy_tensor)
        x=self.flatten(x)
        x=self.dropout(x)
        #x=self.hid_layer(x)
        #x=self.dropout2(x)
        #attn_output, attn_output_weights = multihead_attn(query, key, value)
        x=self.out_layer(x)
        yhat=self.sm(x)

        #N 27 128  switch axes 27 N 128 --> 0 N 128

        return x, yhat

In [ ]:
#Add A Positional Encoding modified from torch website

from torch import nn, Tensor
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x=torch.swapaxes(x,0,1)
        x = x + self.pe[:x.size(0)]
        x=self.dropout(x)
        x=torch.swapaxes(x,0,1)
        return x

In [ ]:
#Preparing Training Data w Embeddings
#After we have tokenized and shuffled batches of our training data, we need to prepare data for training in the LSTM,
#by converting each word in the tokenized text to a 300d embedding.
#We use the Clinical Embeddings here: https://github.com/gweissman/clinical_embeddings/blob/master/README.md.

def prepareXY(batchsize,batch):
        #x = torch.from_numpy(np.zeros((batchsize,WORDlen)))
        #x = torch.from_numpy(np.zeros((batchsize,WORDlen,300)))

        x=(-2)*torch.rand((batchsize,WORDlen,300))+1

        x2=np.array(batch['Text']).T
        mask=batch['Mask']

        for i2 in range(batchsize):
          for j2 in range(WORDlen): #27 --> 206

            #x[i2,j2] = vocab_list[x2[i2][j2]] #"the"->50
            word=x2[i2][j2]
            if modelemb.wv.has_index_for(word):
              x[i2,j2]=torch.from_numpy(modelemb.wv.get_vector(word))

            else:
              #x[i2,j2]=torch.zeros(300)
              x[i2,j2]=(-2)*torch.rand(300)+1 #SEARCH dict for closest word and add its embedding word2vec embedding -> take dot product with all the other words to find most similar word -> use its word embedding


        #x = torch.tensor(x).to(torch.int64)
        x = torch.tensor(x).to(torch.float32)
        y = np.array(batch['Class'])
        y = torch.from_numpy(y.astype(int))

        yactual = []
        for j in range(batchsize):
          one_hot = np.zeros(5)
          one_hot[y[j]] = 1
          yactual.append(one_hot)


        yactual = torch.Tensor(yactual)
        return x,yactual,mask

#Get model accuracy
def accuracyfun(batchsize,q,yactual,include=False,batch=None):
        accuracy=0
        for i in range(batchsize):
          if torch.argmax(q[i]) == torch.argmax(yactual[i]):
            accuracy+=1
          else:
            if include:
              print("exp","true",torch.argmax(q[i]),torch.argmax(yactual[i]))
              print(batch[i])

        accuracy/=batchsize
        return accuracy


In [ ]:
#Transformer Training Loop

loss_fn = nn.CrossEntropyLoss()
model = MyNeuralNet()

#model.load_state_dict(torch.load('transmodel.pkl'))
learning_rate = 1e-3
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
trainlosses,testlosses,testaccuracies,trainaccuracies = [],[],[],[]

#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
#model.to(device)

for epoch in range(40):
    for (idx, batch) in enumerate(trainloader):
    # Print the 'text' data of the batch

    # Print the 'class' data of batch
        if idx>80: break
        x,yactual,mask = prepareXY(64,batch)

        """
        mask=torch.zeros((64,WORDlen))

        for ix in range(len(x)):
          row=x[ix]
          for elem in range(len(row)):
            if torch.equal(row[elem],torch.zeros(300)):
              mask[ix,elem]=1
        print(mask)
        """

        p,q=model(x,mask)

        loss =loss_fn(p,yactual)
        accuracy = accuracyfun(64,q,yactual)

        #print(model.state_dict())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    for idx, batch in enumerate(trainloader2):


        x,yactual,mask = prepareXY(2000,batch)
        p,q=model(x,mask)
        loss =loss_fn(p,yactual)

        accuracy = accuracyfun(2000,q,yactual)
        counts=[0,0,0,0,0]

        for i in range(2000):
          counts[torch.argmax(q[i])-1]+=1


        print(counts)
        break

    for idx, batch in enumerate(testloader2):
        x,yactual,mask = prepareXY(2000,batch)
        p,q=model(x,mask)
        testloss =loss_fn(p,yactual)

        testaccuracy=accuracyfun(2000,q,yactual)

        break

    torch.save(model.state_dict(), 'transmodelnewtokenizer50-2.pkl')
    trainlosses.append(loss.item())
    testlosses.append(testloss.item())
    testaccuracies.append(testaccuracy)
    trainaccuracies.append(accuracy)
    print(loss.item(),accuracy,testloss.item(),testaccuracy)
    #scheduler.step()

print(trainlosses,testlosses)
print(testaccuracies,trainaccuracies)


In [ ]:
#model = MyNeuralNet()
#model.load_state_dict(torch.load('/content/transmodelnewtokenizer50-2.pkl'))
loss_fn = nn.CrossEntropyLoss()

from sklearn.metrics import confusion_matrix
import numpy as np

#cm = confusion_matrix(y_true, y_pred)
#print(cm)

for idx, batch in enumerate(testloader2):
        size=2000
        x,yactual,mask = prepareXY(size,batch)

        p,q=model(x,mask)
        testloss =loss_fn(p,yactual)

        testaccuracy=accuracyfun(size,q,yactual)
        print(testaccuracy)
        truelabs=yactual
        L=[]
        for i in range(size):
          L.append(torch.argmax(q[i]))

        L=np.array(L,dtype=int)
        print(yactual.shape)
        print(type(L[0]),type(np.array(yactual,dtype=int)[0]))
        print(confusion_matrix(np.array(torch.argmax(yactual,axis=1)), L))
        testaccuracy=accuracyfun(size,q,yactual,True,np.array(batch['Text']).T)

        break